# Funcionamiento del modelo de seguimiento de aprendizaje
- [Introducción](#introducción)
- [Metodología](#metodología)
- [Resultados](#resultados)
- [Conclusión](#conclusión)

## Entrenamiento del modelo base

Cargamos y declaramos el modelo de seguimiento de aprendizaje.

In [1]:
from src.student_eval.app.evaluation.models import StudentModel
from pyBKT.models import Model
model = StudentModel()

Podedmos ver el tipo de datos que necesita para realizar una primera actualización de los datos de usuarios:

In [2]:
help(model.update_dataset)

Help on method update_dataset in module src.student_eval.app.evaluation.models:

update_dataset(order_id: List[int], user_id: List[str], skill_name: List[str], correct: List[int], item_id: List[str], subject_id: List[str]) method of src.student_eval.app.evaluation.models.StudentModel instance
    Actualiza el dataset (de existir) y entrena el modelo.
    
    Args:
        order_id (list[str]): Descripción del índice de orden único de cada pregunta (item_id) respondida por un estudiante. No se puede repetir entre datasets. Ej: numerical_item_id + timestamp.
        user_id (list[str]): ID del estudiante.
        skill_name (list[str]): Nombre de la habilidad.
        correct (list[int]): 1 si la respuesta es correcta, 0 si es incorrecta, -1 si no se ha respondido.
        item_id (list[str]): ID de la pregunta. Se puede repetir indicando que se ha hecho varias veces la misma pregunta.
    
    Returns:
        dict: Diccionario con los estados de los estudiantes y habilidades.
        

Generamos datos de prueba para entrenar el modelo BKT:

In [2]:
data = {"order_id" : [1, 2, 3, 4, 1, 2, 3, 4, 1, 2, 3, 4],
        "user_id": ["Bob", "Bob", "Bob", "Bob", "Alice", "Alice", "Alice", "Alice", "Charlie", "Charlie", "Charlie", "Charlie"],
        "skill_name": ["equations", "equations", "lens", "lens", "lens", "algebra", "algebra", "lens", "lens", "algebra", "algebra", "lens"],
        "correct": [1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1],
        "item_id": ["EQ1", "EQ2", "LE1", "LE2", "LE1", "ALG1","ALG2", "LE2","LE1", "ALG1","ALG2", "LE2"],
        "subject_id": ["math", "math", "physics", "physics","physics", "math", "math", "physics", "physics", "math", "math", "physics"],
        }

Declaramos las rutas de guardado del modelo y los datos de prueba y lo entrenamos con los datos generados:


In [3]:
# Creamos directorio temporal
import os

if os.path.exists("tmp"):
    # Limpiar el directorio temporal
    import shutil
    shutil.rmtree("tmp")

os.makedirs("tmp", exist_ok=True)
os.makedirs("tmp/default_csv", exist_ok=True)
os.makedirs("tmp/default_model", exist_ok=True)

CSV_PATH =  os.path.join("tmp", "default_csv", "student_eval.csv")
MODEL_PATH = os.path.join("tmp", "default_model")



# Estas rutas se pueden declarar en models.py para evitar definirlas cada vez 
model.csv_path = CSV_PATH
model.model_path = MODEL_PATH 

In [4]:
# Entrenamos el modelo con update_dataset
results_training = model.update_dataset(**data)

In [5]:
import pprint
# Resultados iniciales
print("Resultados iniciales")
print("=====================================")
pprint.pprint(results_training["students_states"])
print("=====================================")
pprint.pprint(results_training["skills_states"])

Resultados iniciales
{'Alice': {'forgets': {'math': {'algebra': 0.0}, 'physics': {'lens': 0.0}},
           'learns': {'math': {'algebra': 0.31732635370004325},
                      'physics': {'lens': 0.012254688940361948}}},
 'Bob': {'forgets': {'math': {'equations': 0.0}, 'physics': {'lens': 0.0}},
         'learns': {'math': {'equations': 0.7123999882996016},
                    'physics': {'lens': 0.012254688940361948}}},
 'Charlie': {'forgets': {'math': {'algebra': 0.0}, 'physics': {'lens': 0.0}},
             'learns': {'math': {'algebra': 0.05401202078543974},
                        'physics': {'lens': 0.8818434926410905}}}}
{'math': {'algebra': {'forgets': 0.493645049727295,
                      'guesses': 0.411640006283352,
                      'learns': 0.1558171853015925,
                      'prior': 0.0,
                      'slips': 0.15369979107280193},
          'equations': {'forgets': 0.2327070155603201,
                        'guesses': 1.0,
                 

Podemos añadir más datos usando la misma función. Estos se anexionarán a los datos anteriores y se entrenará el modelo con todos los datos.

In [6]:
new_data = {"order_id" : [5, 6, 7, 5, 6, 7, 5, 6, 7],
        "user_id": ["Bob", "Bob", "Bob", "Alice", "Alice", "Alice", "Charlie", "Charlie", "Charlie"],
        "skill_name": ["equations", "equations", "lens", "lens", "lens", "algebra", "circuits", "circuits", "circuits"],
        "correct": [1, 1, 1, 1, 1, 0, 0, 1, 0],
        "item_id": ["EQ3", "EQ4", "LE3", "LE3", "LE4", "ALG3","C1","C2", "C3"],
        "subject_id": ["math", "math", "physics", "physics","physics", "math","physics","physics","physics"],
        }

In [7]:
# Entrenamos el modelo con update_dataset
results_training = model.update_dataset(**new_data)

In [8]:
import pprint
# Resultados tras ampliar
print("Resultados actualizados")
print("=====================================")
pprint.pprint(results_training["students_states"])
print("=====================================")
pprint.pprint(results_training["skills_states"])

Resultados actualizados
{'Alice': {'forgets': {'math': {'algebra': 0.0}, 'physics': {'lens': 0.0}},
           'learns': {'math': {'algebra': 0.2767342764856434},
                      'physics': {'lens': 0.05382961058969178}}},
 'Bob': {'forgets': {'math': {'equations': 0.0}, 'physics': {'lens': 0.0}},
         'learns': {'math': {'equations': 0.7301626954864259},
                    'physics': {'lens': 0.05162563634100992}}},
 'Charlie': {'forgets': {'math': {'algebra': 0.0},
                         'physics': {'circuits': 0.0, 'lens': 0.0}},
             'learns': {'math': {'algebra': 0.06048517256698108},
                        'physics': {'circuits': 0.36594456482115256,
                                    'lens': 0.8634234532922855}}}}
{'math': {'algebra': {'forgets': 0.7317544528152401,
                      'guesses': 0.3399389507232356,
                      'learns': 0.1185245610735145,
                      'prior': 0.0,
                      'slips': 0.2784104029941059},


## Seguimiento en tiempo real

Podemos realizar una actualización en tiempo real de un estudiante en base a preguntas para determinadas skills.

In [9]:
# Datos de configuración para la evaluación en tiempo real
setup_data = {"user_id":"Bob",
              "skill_names":["equations", "lens"]}


# Asignamos los paths necesarios:
evaluation_csv_path_trained = os.path.join("tmp", "rte_trained")
evaluation_csv_path_non_trained = os.path.join("tmp", "rte_non_trained")
evaluation_path = os.path.join("tmp", "rte")
if not os.path.exists(evaluation_csv_path_trained):
    os.makedirs(evaluation_csv_path_trained, exist_ok=True)
if not os.path.exists(evaluation_csv_path_non_trained):
    os.makedirs(evaluation_csv_path_non_trained, exist_ok=True)
if not os.path.exists(evaluation_path):
    os.makedirs(evaluation_path, exist_ok=True)

model.evaluation_csv_path_non_trained = os.path.join(evaluation_csv_path_trained ,"student_eval_non_trained.csv")
model.evaluation_csv_path_trained = os.path.join(evaluation_csv_path_non_trained, "student_eval_trained.csv")
model.evaluation_path = evaluation_path


# Iniciamos la evaluación en tiempo real
response = model.start_real_time_evaluation(**setup_data)
print(response)

{'status': 'ok', 'message': 'Rosters creados/reutilizados por skill', 'roster_paths': {'equations': 'tmp/rte/roster_Bob_equations_lens.pkl', 'lens': 'tmp/rte/roster_Bob_equations_lens.pkl'}}


In [10]:
import numpy as np

# Preparamos para la generación de respuestas ficticias
order_id_init_le = 8
order_id_init_eq = 8
user_id = "Bob"
item_id_base_eq = "EQ"
item_id_base_le = "LE"
n_item_eq = 5
n_item_le = 5
subject_id_eq = "math"
subject_id_le = "physics"
# corrects = [1, 0, 1, 1, 1, 0, 1, -1 ,1 ,1, 1, 1, 1, 1, 1, 1, 1, 1]
n_tries = 20
for i in range(n_tries):
  if i % 2 == 0:
    order_id = order_id_init_eq + i
    item_id = item_id_base_eq + str(n_item_eq)
    subject_id = subject_id_eq
    skill_name = "equations"
    roster_path = response["roster_paths"]["equations"]
    n_item_eq += 1
  else:
    order_id = order_id_init_le + i
    item_id = item_id_base_le + str(n_item_le)
    subject_id = subject_id_le
    skill_name = "lens"
    roster_path = response["roster_paths"]["lens"]
    n_item_le += 1
  # Generamos una respuesta ficticia
  correct = np.random.randint(2)
  studentresult = model.real_time_evaluation(
      order_id=order_id,
      user_id=user_id,
      skill_name=skill_name,
      correct=correct,
      item_id=item_id,
      subject_id=subject_id,
      roster_path=roster_path,
  )
  print("Skill: ", skill_name)
  print(studentresult)
  print("\n")
  

Skill:  equations
{'state': 'UNMASTERED', 'correct_prob': 1.0, 'state_prob': 0.768072269631725}


Skill:  lens
{'state': 'UNMASTERED', 'correct_prob': 0.6110895878862552, 'state_prob': 0.6288508601514526}


Skill:  equations
{'state': 'UNMASTERED', 'correct_prob': 1.0, 'state_prob': 0.7700405101632691}


Skill:  lens
{'state': 'UNMASTERED', 'correct_prob': 0.8495217882130365, 'state_prob': 0.9368142873409361}


Skill:  equations
{'state': 'UNMASTERED', 'correct_prob': 1.0, 'state_prob': 0.7701426999366273}


Skill:  lens
{'state': 'MASTERED', 'correct_prob': 0.8741077901622134, 'state_prob': 0.9685700208818699}


Skill:  equations
{'state': 'UNMASTERED', 'correct_prob': 1.0, 'state_prob': 0.7701480055634609}


Skill:  lens
{'state': 'MASTERED', 'correct_prob': 0.8758801455511787, 'state_prob': 0.9708592277886361}


Skill:  equations
{'state': 'UNMASTERED', 'correct_prob': 1.0, 'state_prob': 0.770148281028169}


Skill:  lens
{'state': 'MASTERED', 'correct_prob': 0.876004066162892, 'stat

Podemos actualizar el modelo con los nuevos datos generados.

In [11]:
model.update_dataset_evaluation()

{'students_states': {'Alice': {'learns': {'physics': {'lens': 0.008973475260650598},
    'math': {'algebra': 0.2767342764856434}},
   'forgets': {'physics': {'lens': 0.0}, 'math': {'algebra': 0.0}}},
  'Bob': {'learns': {'physics': {'lens': 0.008997797383850068},
    'math': {'equations': 0.7150025154969799}},
   'forgets': {'physics': {'lens': 0.0}, 'math': {'equations': 0.0}}},
  'Charlie': {'learns': {'physics': {'lens': 0.9272402185371594,
     'circuits': 0.36594456482115256},
    'math': {'algebra': 0.06048517256698108}},
   'forgets': {'physics': {'lens': 0.0, 'circuits': 0.0},
    'math': {'algebra': 0.0}}}},
 'skills_states': {'math': {'algebra': {'prior': 0.0,
    'learns': 0.1185245610735145,
    'guesses': 0.3399389507232356,
    'slips': 0.2784104029941059,
    'forgets': 0.7317544528152401},
   'equations': {'prior': 0.0,
    'learns': 0.3890535306106614,
    'guesses': 0.3275298845691227,
    'slips': 0.01115313929337402,
    'forgets': 0.47546158030702645}},
  'physics'